In [7]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000  # 5 seconds

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: 'Arial', sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    height: 100vh;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .back-button svg {{
    width: 26px;
    height: 26px;
    fill: none;
    stroke: currentColor;
    stroke-width: 2.5;
    stroke-linecap: round;
    stroke-linejoin: round;
    transition: transform 0.3s ease;
  }}

  .back-button:hover {{
    background: rgba(0, 255, 255, 0.15);
    color: #00bcd4;
    transform: scale(1.05);
    box-shadow: 0 0 10px rgba(0, 255, 255, 0.6);
  }}

  .back-button:hover svg {{
    transform: translateX(-3px);
  }}

  img {{
    max-height: 90vh;
    max-width: 90vw;
    object-fit: contain;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    border-radius: 6px;
  }}

  .controls {{
    margin-top: 16px;
    display: flex;
    gap: 12px;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  button.control:hover {{
    background: #374151;
  }}

  .counter {{
    margin-top: 8px;
    font-size: 13px;
    color: #9ca3af;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<img id="slide" src="{images[0]}" alt="slideshow image">

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="next()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<script>
const images = {images};
let index = 0;
let timer = null;
let auto = true;

const slide = document.getElementById("slide");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");

function update() {{
  slide.src = images[index];
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function next() {{
  index = (index + 1) % images.length;
  update();
}}

function prev() {{
  index = (index - 1 + images.length) % images.length;
  update();
}}

function startAuto() {{
  if (timer) return;   // 🔒 only start once
  timer = setInterval(() => {{
    if (auto) next();
  }}, {AUTO_ADVANCE_MS});
}}

function pauseAuto() {{
  auto = false;
  toggleBtn.textContent = "▶ Resume";
}}

function resumeAuto() {{
  auto = true;
  toggleBtn.textContent = "⏸ Pause";
}}

function toggleAuto() {{
  auto ? pauseAuto() : resumeAuto();
}}

document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") next();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

update();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with {len(images)} images")


Generated index.html with 6 images


## Let's make one with an animation!

In [17]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .slideshow {{
    position: relative;
    width: 100vw;
    height: calc(100vh - 120px);
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  .slide {{
    position: absolute;
    max-width: 90vw;
    max-height: 90vh;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    transition: transform {ANIM_MS}ms ease;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  .controls {{
    position: fixed;
    bottom: 80px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 12px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  .counter {{
    position: fixed;
    bottom: 60px;
    width: 100%;
    text-align: center;
    font-size: 13px;
    color: #9ca3af;
  }}

  /* Thumbnail strip */
  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 60px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;   /* <-- center thumbnails */
    gap: 8px;
    padding: 6px;
    box-sizing: border-box;
    overflow-x: auto;
    scroll-behavior: smooth;
  }}


  .thumb-strip img {{
    height: 48px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
const images = {images};
let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => jumpTo(i);
  thumbs.appendChild(t);
}});

// Create initial slide
let current = document.createElement("img");
current.src = images[0];
current.className = "slide center";
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = document.createElement("img");
  incoming.src = images[newIndex];
  incoming.className = "slide " + (direction === "next" ? "off-right" : "off-left");
  slideshow.appendChild(incoming);

  incoming.getBoundingClientRect();

  current.className = "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
}}

function startAuto() {{
  if (timer) return;
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
}}

// Touch swipe support
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""


with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)


## Let's make it a nice gallery

In [20]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .slideshow {{
    position: relative;
    width: 100vw;
    height: calc(100vh - 120px);
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  .slide {{
    position: absolute;
    max-width: 90vw;
    max-height: 90vh;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    transition: transform {ANIM_MS}ms ease;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  .controls {{
    position: fixed;
    bottom: 80px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 12px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  .counter {{
    position: fixed;
    bottom: 60px;
    width: 100%;
    text-align: center;
    font-size: 13px;
    color: #9ca3af;
  }}

  /* Centered thumbnail strip */
  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 60px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    gap: 8px;
    padding: 6px;
    box-sizing: border-box;
    overflow-x: auto;
    scroll-behavior: smooth;
  }}

  .thumb-strip img {{
    height: 48px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
const images = {images};
let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Create initial slide
let current = document.createElement("img");
current.src = images[0];
current.className = "slide center";
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = document.createElement("img");
  incoming.src = images[newIndex];
  incoming.className = "slide " + (direction === "next" ? "off-right" : "off-left");
  slideshow.appendChild(incoming);

  incoming.getBoundingClientRect();

  current.className = "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe support
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)


### Let's make the images clickable links

In [21]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .slideshow {{
    position: relative;
    width: 100vw;
    height: calc(100vh - 120px);
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  /* The slide is the LINK wrapper */
  .slide {{
    position: absolute;
    max-width: 90vw;
    max-height: 90vh;
    display: flex;
    align-items: center;
    justify-content: center;
    transition: transform {ANIM_MS}ms ease;
  }}

  .slide img {{
    max-width: 100%;
    max-height: 100%;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    display: block;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  .controls {{
    position: fixed;
    bottom: 80px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 12px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  .counter {{
    position: fixed;
    bottom: 60px;
    width: 100%;
    text-align: center;
    font-size: 13px;
    color: #9ca3af;
  }}

  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 60px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    gap: 8px;
    padding: 6px;
    box-sizing: border-box;
    overflow-x: auto;
    scroll-behavior: smooth;
  }}

  .thumb-strip img {{
    height: 48px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
const images = {images};

// One URL per image — edit these as needed.
// By default, they’re all the same placeholder.
const urls = images.map((_, i) => "https://example.com/placeholder-" + (i + 1));

let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Helper: create a linked slide
function createSlide(src, href, className) {{
  const link = document.createElement("a");
  link.href = href;
  link.target = "_blank";
  link.className = className;

  const img = document.createElement("img");
  img.src = src;

  link.appendChild(img);
  return link;
}}

// Create initial slide
let current = createSlide(images[0], urls[0], "slide center");
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = createSlide(
    images[newIndex],
    urls[newIndex],
    "slide " + (direction === "next" ? "off-right" : "off-left")
  );
  slideshow.appendChild(incoming);

  // Force layout
  incoming.getBoundingClientRect();

  current.className =
    "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
  resetTimer();
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)


In [23]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

# Build JS arrays
images_js = "[" + ",".join(f"'{img}'" for img in images) + "]"
urls_js = "[" + ",".join("'http://meritpages.com/michael_woodcock'" for _ in images) + "]"

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .back-button svg {{
    width: 26px;
    height: 26px;
    fill: none;
    stroke: currentColor;
    stroke-width: 2.5;
    stroke-linecap: round;
    stroke-linejoin: round;
    transition: transform 0.3s ease;
  }}

  .back-button:hover {{
    background: rgba(0, 255, 255, 0.15);
    color: #00bcd4;
    transform: scale(1.05);
    box-shadow: 0 0 10px rgba(0, 255, 255, 0.6);
  }}

  .back-button:hover svg {{
    transform: translateX(-3px);
  }}

  .slideshow {{
    position: relative;
    width: 100vw;
    height: calc(100vh - 120px);
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  /* The slide is the LINK wrapper */
  .slide {{
    position: absolute;
    max-width: 90vw;
    max-height: 90vh;
    display: flex;
    align-items: center;
    justify-content: center;
    transition: transform {ANIM_MS}ms ease;
  }}

  .slide img {{
    max-width: 100%;
    max-height: 100%;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    display: block;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  .controls {{
    position: fixed;
    bottom: 80px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 12px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  button.control:hover {{
    background: #374151;
  }}

  .counter {{
    position: fixed;
    bottom: 60px;
    width: 100%;
    text-align: center;
    font-size: 13px;
    color: #9ca3af;
  }}

  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 60px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    gap: 8px;
    padding: 6px;
    box-sizing: border-box;
    overflow-x: auto;
    scroll-behavior: smooth;
  }}

  .thumb-strip img {{
    height: 48px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
// Auto-generated by Python
const images = {images_js};
const urls = {urls_js};  // one URL per image (edit in JS if needed)

let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Helper: create a linked slide
function createSlide(src, href, className) {{
  const link = document.createElement("a");
  link.href = href;
  link.target = "_blank";
  link.className = className;

  const img = document.createElement("img");
  img.src = src;

  link.appendChild(img);
  return link;
}}

// Create initial slide
let current = createSlide(images[0], urls[0], "slide center");
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = createSlide(
    images[newIndex],
    urls[newIndex],
    "slide " + (direction === "next" ? "off-right" : "off-left")
  );
  slideshow.appendChild(incoming);

  // Force layout
  incoming.getBoundingClientRect();

  current.className =
    "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
  resetTimer();
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")

Generated index.html with animated sliding transitions (6 images)


## Try to make sure the full pic always shows

In [1]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

# Build JS arrays
images_js = "[" + ",".join(f"'{img}'" for img in images) + "]"
urls_js = "[" + ",".join("'http://meritpages.com/michael_woodcock'" for _ in images) + "]"

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .back-button svg {{
    width: 26px;
    height: 26px;
    fill: none;
    stroke: currentColor;
    stroke-width: 2.5;
    stroke-linecap: round;
    stroke-linejoin: round;
    transition: transform 0.3s ease;
  }}

  .back-button:hover {{
    background: rgba(0, 255, 255, 0.15);
    color: #00bcd4;
    transform: scale(1.05);
    box-shadow: 0 0 10px rgba(0, 255, 255, 0.6);
  }}

  .back-button:hover svg {{
    transform: translateX(-3px);
  }}

  /* FULL-VIEWPORT SLIDESHOW */
  .slideshow {{
    position: relative;
    width: 100vw;
    height: 100vh; /* always full screen */
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  /* Slide wrapper */
  .slide {{
    position: absolute;
    max-width: 100vw;
    max-height: 100vh;
    display: flex;
    align-items: center;
    justify-content: center;
    transition: transform {ANIM_MS}ms ease;
  }}

  /* Image ALWAYS fits fully with 10px padding */
  .slide img {{
    width: calc(100% - 20px);   /* 10px padding left + right */
    height: calc(100% - 20px);  /* 10px padding top + bottom */
    margin: 10px;               /* actual spacing */
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    display: block;
    cursor: zoom-in;
    transition: transform 0.3s ease;
  }}

  /* Zoomed mode */
  .slide img.zoomed {{
    transform: scale(1.8);
    cursor: zoom-out;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  .controls {{
    position: fixed;
    bottom: 80px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 12px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 10px 16px;
    font-size: 14px;
    border-radius: 6px;
    cursor: pointer;
  }}

  button.control:hover {{
    background: #374151;
  }}

  .counter {{
    position: fixed;
    bottom: 60px;
    width: 100%;
    text-align: center;
    font-size: 13px;
    color: #9ca3af;
  }}

  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 60px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    gap: 8px;
    padding: 6px;
    box-sizing: border-box;
    overflow-x: auto;
    scroll-behavior: smooth;
  }}

  .thumb-strip img {{
    height: 48px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
// Auto-generated by Python
const images = {images_js};
const urls = {urls_js};

let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Helper: create a linked slide
function createSlide(src, href, className) {{
  const link = document.createElement("a");
  link.href = href;
  link.target = "_blank";
  link.className = className;

  const img = document.createElement("img");
  img.src = src;

  link.appendChild(img);
  return link;
}}

// Create initial slide
let current = createSlide(images[0], urls[0], "slide center");
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = createSlide(
    images[newIndex],
    urls[newIndex],
    "slide " + (direction === "next" ? "off-right" : "off-left")
  );
  slideshow.appendChild(incoming);

  incoming.getBoundingClientRect();

  current.className =
    "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
  resetTimer();
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

// CLICK TO ZOOM
document.addEventListener("click", (e) => {{
  if (e.target.tagName === "IMG" && e.target.closest(".slide")) {{
    e.target.classList.toggle("zoomed");
  }}
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)
